# Merging Shards

## Data

### Why is data in shards?

The data Is in shards because the API would not let us continuously hit it even with long intervals, so the fetching script was changed to resemble mapper reducer. We manually implemented a job system and these jobs could run on different networks at the same time. 

### What the data looks like?

Data given by the API comes with six lists in which two of them are just hashes, which can be dropped without any problems. There is a list which is titled "for you", which we think is about specific recommendations for a user. We also drop that.

We combine listings by IDs buildings with multiple apartments have different IDs And if the complete house is available for rent, it has a different ID attribute.

Each apartment will be treated as a separate listing.

In [32]:
import pandas as pd
import json
import pathlib
from typing import Callable, TypeVar, Optional
from collections.abc import Iterable
import matplotlib.pyplot as plt
import seaborn as sns

In [155]:
def process_single_file(path:pathlib.Path):
    with open(path, 'r') as f:
        data = json.load(f)
        all_entries = []

    for key, value in data.items():
        if key == "mapResults" or key == 'listResults':
            all_entries.extend(value)

    filtered_entries = [
        entry for entry in all_entries if 'detailUrl' in entry and 'homedetails' in entry['detailUrl']
    ]

    my_df = pd.DataFrame(filtered_entries)

    return my_df

In [156]:
big_df = pd.DataFrame()

shards = pathlib.Path('Data/Data Shards')
for x in shards.glob("**/*.json"):
    temp_df = process_single_file(x)
    big_df = pd.concat([big_df, temp_df], ignore_index=True)


In [157]:
big_df.head()

,zpid,id,rawHomeStatusCd,marketingStatusSimplifiedCd,providerListingId,imgSrc,hasImage,detailUrl,statusType,statusText,...,canSaveBuilding,availabilityCount,isInstantTourEnabled,isContactable,plid,minBeds,minBaths,buildingId,unitCount,minArea
0,2081331532,2081331532,ForRent,For Rent,g3qqbttyw1g7,https://photos.zillowstatic.com/fp/0f678de9f33...,True,https://www.zillow.com/homedetails/498-Jeffers...,FOR_RENT,Apartment for rent,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,440689351,440689351,ForRent,For Rent,4msp054rpy0w3,https://photos.zillowstatic.com/fp/e54dce7afc0...,True,https://www.zillow.com/homedetails/371-Koscius...,FOR_RENT,Apartment for rent,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,442762524,442762524,ForRent,For Rent,56cyrsnd6f4z1,https://photos.zillowstatic.com/fp/2eab327f7e9...,True,https://www.zillow.com/homedetails/373-Koscius...,FOR_RENT,Apartment for rent,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2077973303,2077973303,ForRent,For Rent,1v8znm7bxz78f,https://photos.zillowstatic.com/fp/05657172691...,True,https://www.zillow.com/homedetails/48-Jefferso...,FOR_RENT,Apartment for rent,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,30629385,30629385,ForRent,For Rent,5at3ufj2pccje,https://photos.zillowstatic.com/fp/94b3e03c781...,True,https://www.zillow.com/homedetails/573-Evergre...,FOR_RENT,Apartment for rent,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Duplicates

There might be duplicates as zillow mihgt give us properties which spanned multiple zipcodes or happened to be on edge of zip codes. We drop duplicated so there is only one listing per property

In [158]:
if big_df['zpid'].duplicated().any():
    print("Duplicate addresses found!")
else:
    print("No duplicate addresses.")

Duplicate addresses found!


In [159]:
print(f"We had {big_df.shape[0]} entries before removing duplicates.")
big_df = big_df.drop_duplicates(subset='zpid', keep='first')
print(f"We have {big_df.shape[0]} entries after removing duplicates.")

We had 46796 entries before removing duplicates.
We have 18254 entries after removing duplicates.


### Unique ZillowID but same Addresses what??

We find that there are some listings without a disclosed address. These would be lost if we would have used uniqeu addresses

In [160]:
duplicates_by_address = big_df[big_df.duplicated(subset=['address'], keep=False)]
print("Duplicates by Address:")
duplicates_by_address.head()

Duplicates by Address:


,zpid,id,rawHomeStatusCd,marketingStatusSimplifiedCd,providerListingId,imgSrc,hasImage,detailUrl,statusType,statusText,...,canSaveBuilding,availabilityCount,isInstantTourEnabled,isContactable,plid,minBeds,minBaths,buildingId,unitCount,minArea
428,2100634408,2100634408,ForRent,For Rent,1szmn2kdvd58h,https://photos.zillowstatic.com/fp/5a8d7af6094...,True,https://www.zillow.com/homedetails/West-Harris...,FOR_RENT,Apartment for rent,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
431,442405589,442405589,ForRent,For Rent,uf51mw0t307h,https://photos.zillowstatic.com/fp/c8c28aeb8db...,True,https://www.zillow.com/homedetails/West-Harris...,FOR_RENT,Apartment for rent,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
432,2056770390,2056770390,ForRent,For Rent,ksqhwbp2kzrv,https://photos.zillowstatic.com/fp/1066ac625cc...,True,https://www.zillow.com/homedetails/White-Plain...,FOR_RENT,Apartment for rent,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
434,2083184622,2083184622,ForRent,For Rent,5ccze5nmyv024,https://photos.zillowstatic.com/fp/6125d7d52e9...,True,https://www.zillow.com/homedetails/White-Plain...,FOR_RENT,Apartment for rent,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
441,2094562201,2094562201,ForRent,For Rent,35ukrtqwvxsv,https://photos.zillowstatic.com/fp/2508de21cac...,True,https://www.zillow.com/homedetails/White-Plain...,FOR_RENT,Apartment for rent,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# ZIP-CODE DF Changes

In [161]:
zip_code_df = pd.read_csv('../ny_rent_prediction_data_comprehensive.csv')

In [ ]:
# Look for missing values
print('total null values \n', zip_code_df.isnull().sum())

# Look for weird or outlier values in each column
for column in zip_code_df.columns:
  print(f"\nExploring column: {column}")
  print(zip_code_df[column].value_counts())
  if pd.api.types.is_numeric_dtype(zip_code_df[column]):
    plt.figure()
    zip_code_df[column].hist()
    plt.title(f"Histogram of {column}")
    plt.show()

In [162]:
rows_with_negative_666666 = zip_code_df[(zip_code_df == -666666666).any(axis=1)]

zip_code_df_cleaned = zip_code_df[(zip_code_df != -666666666).all(axis=1)]

print("Dropped rows:")
print(rows_with_negative_666666)

Dropped rows:
     Total Population   Median Age  Male Population  Female Population  \
17                  0 -666666666.0                0                  0   
34              19138         39.3             8426              10712   
48                  0 -666666666.0                0                  0   
118                 0 -666666666.0                0                  0   
130                 0 -666666666.0                0                  0   

     White Alone  Black or African American Alone  Asian Alone  \
17             0                                0            0   
34          2544                            12593          731   
48             0                                0            0   
118            0                                0            0   
130            0                                0            0   

     Hispanic or Latino  Number of Households  Median Household Income  ...  \
17                    0                     0               -6666

In [163]:
zip_code_df_cleaned.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
Total Population,134.0,54022.746269,26887.244673,3736.0,32704.00,51089.50,76292.000,112750.0
Median Age,134.0,38.315672,4.887853,25.8,34.80,37.55,41.675,51.9
Male Population,134.0,25978.917910,12903.088847,1996.0,16636.25,24825.50,36278.000,59755.0
Female Population,134.0,28043.828358,14115.569858,1740.0,16889.25,27023.00,40046.500,56197.0
White Alone,134.0,20875.261194,15097.630907,1074.0,9402.50,17577.00,28094.750,64293.0
Black or African American Alone,134.0,11954.022388,16392.382049,10.0,1589.75,4396.50,17095.500,81608.0
Asian Alone,134.0,7489.641791,9516.468054,52.0,1536.75,4041.00,9019.750,57749.0
Hispanic or Latino,134.0,16508.164179,16001.235103,203.0,6267.00,9820.50,24694.000,85529.0
Number of Households,134.0,20801.462687,9751.214977,1990.0,13813.00,20766.50,29451.500,41653.0
Median Household Income,134.0,88444.783582,42293.405948,26400.0,59374.25,79367.50,107289.500,250001.0


## Make some DF changes

There are some columns which are not useful for us, for example rawHomeStatusCd, marketingStatusSimplifiedCd, etc.

In [164]:
# Drop rows where the 'price' column has empty values
big_df = big_df.dropna(subset=['price'])

# Verify if rows with empty 'price' are removed
print(f"Shape after removing rows with empty price: {big_df.shape}")
print(big_df['price'].isnull().sum())  # This should return 0

Shape after removing rows with empty price: (18250, 73)
0


In [165]:
big_df['state_hdp'] = big_df['hdpData'].apply(
    lambda x: x.get('homeInfo', {}).get('state') if isinstance(x, dict) else None
)

big_df['addressZipcode'] = big_df['hdpData'].apply(
    lambda x: x.get('homeInfo', {}).get('zipcode') if isinstance(x, dict) else None
)

big_df['addressCity'] = big_df['hdpData'].apply(
    lambda x: x.get('homeInfo', {}).get('city') if isinstance(x, dict) else None
)


print(big_df['state_hdp'].unique())
print('before filtering just NY: ', big_df.shape)
big_df = big_df[big_df['state_hdp'] == 'NY']
print('after filtering just NY: ', big_df.shape)
print(big_df['price'].head())

['NY' 'CT' 'ON' None 'NJ' 'PA' 'ID' 'MA']
before filtering just NY:  (18250, 74)
after filtering just NY:  (18186, 74)
0    $2,449/mo
1    $2,400/mo
2    $2,395/mo
3    $2,600/mo
4    $2,100/mo
Name: price, dtype: object


In [166]:
print(f"Total columns before removing trivially unimportant columns: {big_df.shape[1]}")

big_df_narrowed = big_df.drop(columns=[
    'marketingStatusSimplifiedCd', 'rawHomeStatusCd', 'imgSrc', 'detailUrl',
    'statusType', 'countryCurrency', 'isSaved', 'isUserClaimingOwner',
    'isUserConfirmedClaim', 'pgapt', 'sgapt', 'isShowcaseListing',
    'openHouseStartDate', 'openHouseEndDate', 'isNewYorkState', 'listingType',
    'isFavorite', 'visited', 'rentalMarketingSubType', 'badgeInfo',
    'units', 'lotId', 'isBuilding', 'canSaveBuilding',
    'availabilityCount', 'isInstantTourEnabled', 'isContactable', 'plid',
    'minBeds', 'minBaths', 'buildingId', 'unitCount',
    'minArea', 'isZillowOwned', 'zestimate', 'shouldShowZestimateAsPrice',
    'isHomeRec', 'hasAdditionalAttributions', 'list', 'relaxed',
    'rooms', 'area', 'hasOpenHouse', 'openHouseDescription',
    'priceLabel', 'streetViewURL', 'streetViewMetadataURL'
    ])

print(f"Total columns after removing trivially unimportant columns: {big_df_narrowed.shape[1]}")
big_df_narrowed.head()

Total columns before removing trivially unimportant columns: 74
Total columns after removing trivially unimportant columns: 27


,zpid,id,providerListingId,hasImage,statusText,price,unformattedPrice,address,addressStreet,addressCity,...,hdpData,has3DModel,hasVideo,isFeaturedListing,availabilityDate,brokerName,carouselPhotos,marketingTreatments,timeOnZillow,state_hdp
0,2081331532,2081331532,g3qqbttyw1g7,True,Apartment for rent,"$2,449/mo",2449.0,"498 Jefferson Ave APT 3B, Brooklyn, NY 11221",498 Jefferson Ave APT 3B,Brooklyn,...,"{'homeInfo': {'zpid': 2081331532, 'streetAddre...",False,False,True,2024-11-21 00:00:00,Listing by: Voro Purple LLC,[{'url': 'https://photos.zillowstatic.com/fp/0...,[paid],NaN,NY
1,440689351,440689351,4msp054rpy0w3,True,Apartment for rent,"$2,400/mo",2400.0,"371 Kosciuszko St APT 1, Brooklyn, NY 11221",371 Kosciuszko St APT 1,Brooklyn,...,"{'homeInfo': {'zpid': 440689351, 'streetAddres...",False,False,True,NaN,Listing by: Miracle Capital,[{'url': 'https://photos.zillowstatic.com/fp/e...,[paid],NaN,NY
2,442762524,442762524,56cyrsnd6f4z1,True,Apartment for rent,"$2,395/mo",2395.0,"373 Kosciuszko St #1A, Brooklyn, NY 11221",373 Kosciuszko St #1A,Brooklyn,...,"{'homeInfo': {'zpid': 442762524, 'streetAddres...",False,False,True,2024-11-21 00:00:00,Listing by: Skyhigh Realty NYC LLC,[{'url': 'https://photos.zillowstatic.com/fp/2...,[paid],NaN,NY
3,2077973303,2077973303,1v8znm7bxz78f,True,Apartment for rent,"$2,600/mo",2600.0,"48 Jefferson St #1E, Brooklyn, NY 11206",48 Jefferson St #1E,Brooklyn,...,"{'homeInfo': {'zpid': 2077973303, 'streetAddre...",False,False,True,2024-11-19 00:00:00,Listing by: Nooklyn NYC LLC,[{'url': 'https://photos.zillowstatic.com/fp/0...,[paid],NaN,NY
4,30629385,30629385,5at3ufj2pccje,True,Apartment for rent,"$2,100/mo",2100.0,"573 Evergreen Ave, Brooklyn, NY 11221",573 Evergreen Ave,Brooklyn,...,"{'homeInfo': {'zpid': 30629385, 'streetAddress...",False,False,True,2024-12-01 00:00:00,Listing by: Fifth & Forever LLC,[{'url': 'https://photos.zillowstatic.com/fp/9...,[paid],NaN,NY


### Engineer some trivial attributes

- create houseType from statusText
- break latlong into 2 seperate latitude and longitude attributes
- re-create timeOnZillow Attribute. The oringal attribute is null for many houses however the HDP data for the propeorty seems to have that information so we'll use hdp data for it. if hdp data is not availbel we may fall back to zillow's API
- drop ID columns
- drop Unformatted price because we have regular price
- drop address since it is a composite attribute we have street, city, etc. as independent attribute
- 


In [167]:
def safe_read_subattr(row, attr):
    try:
        return row[attr]
    except (KeyError, TypeError):
        return None

T = TypeVar('T')

def safe_typecast(val, cast: Callable[[object], T]) -> Optional[T]:
    try:
        return cast(val)
    except ValueError:
        return cast()

In [168]:
big_df_narrowed['houseType'] = big_df_narrowed['statusText'].apply(lambda x: x.split(' ')[0] if x is not None else None)
big_df_narrowed.drop(columns=['statusText'], inplace=True)

big_df_narrowed['latitude'] = big_df_narrowed['latLong'].apply(lambda x: safe_read_subattr(x, 'latitude'))
big_df_narrowed['longitude'] = big_df_narrowed['latLong'].apply(lambda x: safe_read_subattr(x, 'longitude'))
big_df_narrowed.drop(columns=['latLong'], inplace=True)

big_df_narrowed['beds'] = big_df_narrowed['beds'].apply(lambda x: safe_typecast(x, int))
big_df_narrowed['carouselPhotos'] = big_df_narrowed['carouselPhotos'].apply(lambda x: len(x) if x is not None and not isinstance(x, float) else 0)
big_df_narrowed['marketingTreatments'] = big_df_narrowed['marketingTreatments'].apply(lambda x: ''.join(x) if isinstance(x, Iterable) else None)
big_df_narrowed['timeOnZillowText'] = big_df_narrowed['variableData'].apply(lambda x: safe_read_subattr(x, 'text'))
big_df_narrowed['daysOnZillowHDP'] = big_df_narrowed['hdpData'].apply(lambda x: safe_typecast(safe_read_subattr(safe_read_subattr(x, 'homeInfo'), 'daysOnZillow'), int))
big_df_narrowed['timeOnZillowHDP'] = big_df_narrowed['hdpData'].apply(lambda x: safe_read_subattr(safe_read_subattr(x, 'homeInfo'), 'timeOnZillow'))

rent_df = big_df_narrowed.drop(columns=[
    'zpid', 'id', 'unformattedPrice', 'address'
])

In [169]:
rent_df.shape

(18186, 27)

In [170]:
rent_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18186 entries, 0 to 46793
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   providerListingId     8756 non-null   object 
 1   hasImage              17676 non-null  object 
 2   price                 18186 non-null  object 
 3   addressStreet         8756 non-null   object 
 4   addressCity           18186 non-null  object 
 5   addressState          8756 non-null   object 
 6   addressZipcode        18172 non-null  object 
 7   isUndisclosedAddress  8756 non-null   object 
 8   beds                  18186 non-null  int64  
 9   baths                 18186 non-null  float64
 10  variableData          18186 non-null  object 
 11  hdpData               18186 non-null  object 
 12  has3DModel            18186 non-null  bool   
 13  hasVideo              18186 non-null  object 
 14  isFeaturedListing     18186 non-null  bool   
 15  availabilityDate      10

In [171]:
# Convert both columns to string type
rent_df['addressZipcode'] = rent_df['addressZipcode'].astype(str)
zip_code_df['zip code tabulation area'] = zip_code_df['zip code tabulation area'].astype(str)

# Now merge
df = pd.merge(rent_df, zip_code_df, 
              left_on='addressZipcode', 
              right_on='zip code tabulation area',
              how='left')

In [172]:
df.columns

Index(['providerListingId', 'hasImage', 'price', 'addressStreet',
       'addressCity', 'addressState', 'addressZipcode', 'isUndisclosedAddress',
       'beds', 'baths', 'variableData', 'hdpData', 'has3DModel', 'hasVideo',
       'isFeaturedListing', 'availabilityDate', 'brokerName', 'carouselPhotos',
       'marketingTreatments', 'timeOnZillow', 'state_hdp', 'houseType',
       'latitude', 'longitude', 'timeOnZillowText', 'daysOnZillowHDP',
       'timeOnZillowHDP', 'Total Population', 'Median Age', 'Male Population',
       'Female Population', 'White Alone', 'Black or African American Alone',
       'Asian Alone', 'Hispanic or Latino', 'Number of Households',
       'Median Household Income', 'Per Capita Income',
       'Population Below Poverty Level', 'Employed', 'Unemployed',
       'Total Income Distribution', 'Median Gross Rent', 'Median Home Value',
       'Occupied Housing Units', 'Vacant Housing Units',
       'Owner-Occupied Units (value < $100,000)', 'Monthly Housing Costs

In [173]:
# Clean and preprocess the data
def clean_price(price_str):
    if isinstance(price_str, str):
        # Remove '$', '/', 'mo', and ',','+','C' characters and convert to float
        return float(price_str.replace('$', '').replace('/mo', '').replace(',', '').replace('+', '').replace('C',''))
    return price_str

In [174]:
# Clean price columns
df['price'] = df['price'].apply(clean_price)
df['Monthly Housing Costs'] = df['Monthly Housing Costs'].apply(clean_price)
df['Median Home Value'] = df['Median Home Value'].apply(clean_price)

In [175]:
df.to_csv('./merged-rent-census.csv')

In [176]:
bins = list(range(0, 10000, 1000)) + [float('inf')]  # Bins from 0 to 10,000, step 500, and one bin for values above 10,000
labels = [f"${i}-{i+999}" for i in range(0, 9000, 1000)] + ["$10,000+"]

# Apply the categorization
df['price_category'] = pd.cut(df['price'], bins=bins, labels=labels, right=False)

# Check the result
print(df[['price', 'price_category']].head())

    price price_category
0  2449.0     $2000-2999
1  2400.0     $2000-2999
2  2395.0     $2000-2999
3  2600.0     $2000-2999
4  2100.0     $2000-2999
